# Encrypting audio into the videos frames and decoding the video back to original state using SNNs

Due to computational power the video has been cropped to one second.

In [1]:
#Downloading pydub
!pip install pip -U
!pip install pydub


Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 31.2 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable


In [36]:
#Importing Libraries
import sys
#accessing where pydub is kept within the container
sys.path.append('/home/ntu-user/.local/lib/python3.10/site-packages')

import nest
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
from IPython.display import display, clear_output
from tqdm import tqdm
import time
from pydub import AudioSegment
import io
from scipy.io import wavfile
import random

## DECODING

The code below defines a function reconstruct_values that recovers the embedded audio data from the encoded image frames. It loads the encoded image, the unique values originally embedded, and the pixel locations where these values were placed. The function then iterates through the encoded image, extracting pixel values between 0 and 9, storing them as recovered values and their corresponding pixel coordinates. It compares the recovered values and pixel locations with the original unique values and selected pixels, reporting any discrepancies. It saves the recovered values to a NumPy file and prints a summary of the reconstruction process, including any errors encountered. The code then iterates through a series of encoded image frames, applying the reconstruct_values function to each, effectively decoding the embedded audio data from the images.

In [177]:
import cv2
import numpy as np
import os

# Paths
reconstructed ='reconstructed'
if not os.path.exists(reconstructed):
    os.makedirs(reconstructed)
encoded_results='encoded_results'

def reconstruct_values(encoded_image_path, unique_values_path, selected_pixels_path, original_image_path, image_name):
    """Reconstructs the hidden values and compares them with the original data."""
    encoded_image = cv2.imread(encoded_image_path, cv2.IMREAD_GRAYSCALE)
    unique_values = np.load(unique_values_path)
    selected_pixels = np.load(selected_pixels_path)

    recovered_values = []
    recovered_pixels = []

    count = 0
    # Find pixels with values 0-9
    for i in range(encoded_image.shape[0]):
        if i % 2 == 0:
            for j in range(encoded_image.shape[1]):
                if 0 <= encoded_image[i, j] <= 9:
                    recovered_values.append(encoded_image[i, j])
                    recovered_pixels.append((i, j))
                    count+=1
    count = 0
    for i in range(encoded_image.shape[0]):
        if i % 2 != 0:
            for j in range(encoded_image.shape[1]):
                if 0 <= encoded_image[i, j] <= 9:
                    recovered_values.append(encoded_image[i, j])
                    recovered_pixels.append((i, j))
                    count+=1

 
    # Compare recovered values with unique values
    value_errors = []
    if len(recovered_values) != len(unique_values):
        value_errors.append(f"Number of recovered values ({len(recovered_values)}) does not match number of unique values ({len(unique_values)}).")
    else:
        for i, val in enumerate(recovered_values):
            if val not in unique_values:
                value_errors.append(f"Recovered value {val} at index {i} not found in unique values.")

    # Compare recovered pixels with selected pixels
    pixel_errors = []
    if len(recovered_pixels) != 1066:
        pixel_errors.append(f"Number of recovered pixels ({len(recovered_pixels)}) does not match number of selected pixels ({1066}).")
    else:
        for i, pixel in enumerate(recovered_pixels):
            if pixel not in selected_pixels:
                pixel_errors.append(f"Recovered pixel {pixel} at index {i} not found in selected pixels.")

    # Print summary
    print(f"Reconstruction Summary for {os.path.basename(encoded_image_path)}:")

    # Print unique and recovered values
    print("  Original Unique Values:", unique_values[0:20])
    print("  Reconstructed Values:", recovered_values[0:20])

    if value_errors:
        print("  Value Errors:")

        for error in value_errors:
            print(f"    {error}")
    else:
        print("  No value errors found.")

    if pixel_errors:
        print("  Pixel Errors:")
        for error in pixel_errors:
            print(f"    {error}")
    else:
        print("  No pixel errors found.")

    np.save(f'reconstructed/reconstructed_audio_{image_name}', recovered_values)

    print("----------------------------------------")
# Reconstruct values for each image
for direction in range(len(sound_data)):
    for frame in range(420, 451): 
        for colour in range(len(colours)):
            image_name = f'{frame}_{colours[colour]}_{side[direction]}'
            encoded_image_path = os.path.join(encoded_results, f'{image_name}_encoded.png')
            unique_values_path = os.path.join(encoded_results, f"{image_name}_unique_test_values.npy")
            selected_pixels_path = os.path.join(encoded_results, f"{image_name}_selected_pixels.npy")
            image, test_vector = reading_frames(float(frame), colour)
            height, width = test_vector.shape
            if direction == 0:
                test_vector = test_vector[:, :width // 2] 
            else:
                test_vector = test_vector[:, width // 2:]
                
            
            reconstruct_values(encoded_image_path, unique_values_path, selected_pixels_path, test_vector, image_name)

            


Processing image: 420.0:Red
Reconstruction Summary for 420_Red_Left_encoded.png:
  Original Unique Values: [7 4 6 0 5 1 5 4 4 2 9 8 6 4 7 7 7 5 7 4]
  Reconstructed Values: [7, 4, 6, 0, 5, 1, 5, 4, 4, 2, 9, 8, 6, 4, 7, 7, 7, 5, 7, 4]
  No value errors found.
  No pixel errors found.
----------------------------------------

Processing image: 420.0:Green
Reconstruction Summary for 420_Green_Left_encoded.png:
  Original Unique Values: [5 4 3 0 6 2 7 2 2 6 9 0 1 0 8 7 5 4 2 1]
  Reconstructed Values: [5, 4, 3, 0, 6, 2, 7, 2, 2, 6, 9, 0, 1, 0, 8, 7, 5, 4, 2, 1]
  No value errors found.
  No pixel errors found.
----------------------------------------

Processing image: 420.0:Blue
Reconstruction Summary for 420_Blue_Left_encoded.png:
  Original Unique Values: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  Reconstructed Values: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  No value errors found.
  No pixel errors found.
----------------------------------------

Processing imag

This code below loads the reconstructed audio data previously found from a series of image frames. For each frame, it loads reconstructed audio data from left and right image sides, separated by color channels (Red, Green, Blue). It then combines the data from each color channel into pairs, creating a list of six values for each pixel (Red even, Red odd, Green even, Green odd, Blue even, Blue odd). These combined lists are then saved as separate NumPy files for the left and right sides of each frame. The code handles an edge case where the last two frames have a different number of data points.

In [178]:
#Loading reconstructed audio for R,G,B and left and right for all frames within the second
for frame in range(420, 451): 
    file_L_Red = np.load(f'reconstructed/reconstructed_audio_{frame}_Red_Left.npy')
    file_L_Green = np.load(f'reconstructed/reconstructed_audio_{frame}_Green_Left.npy')
    file_L_Blue = np.load(f'reconstructed/reconstructed_audio_{frame}_Blue_Left.npy')
    file_R_Red = np.load(f'reconstructed/reconstructed_audio_{frame}_Red_Right.npy')
    file_R_Green = np.load(f'reconstructed/reconstructed_audio_{frame}_Green_Right.npy')
    file_R_Blue = np.load(f'reconstructed/reconstructed_audio_{frame}_Blue_Right.npy')
    odds = 533
    #Frameswhich have 20 values
    if frame == 449 or frame == 450:
        odds = 10
    left_side = []
    right_side = []

    #Iterating through to get list of audios 
    for value in range(0,odds):
        left_side.append([file_L_Red[value], file_L_Red[value+odds], file_L_Green[value], file_L_Green[value+odds], file_L_Blue[value], file_L_Blue[value+odds]]) 
        right_side.append([file_R_Red[value], file_R_Red[value+odds], file_R_Green[value], file_R_Green[value+odds], file_R_Blue[value], file_R_Blue[value+odds]])

    #Saving reconstructed audio
    np.save(f'reconstructed/reconstructed_colours_left_{frame}', left_side)
    np.save(f'reconstructed/reconstructed_colours_right_{frame}', right_side)

The code below iterates through reconstructed audio data from a series of image frames, processing both left and right sides. For each frame and side, it loads the reconstructed data. It then interprets the last digit of the string as a sign (0 for negative, 1 for positive), removes the sign, reverses the remaining digits, removes trailing zeros, converts the result to an integer, and appends it to a correct_order list. Finally, it saves the processed correct_order list as a NumPy file, effectively reconstructing the original numerical audio data.

In [179]:
#Cleaning reconstructed audio
side = ['left', 'right']
#Iterating through direction and frame
for direction in range(2):
    for frame in range(420, 451):
        recon = np.load(f'reconstructed/reconstructed_colours_{side[direction]}_{frame}.npy')

        digits = []
        correct_order= []

        #making list into number
        for digit in recon:
            nums = ''.join(map(str, digit))
            digits.append(nums)

        #Recursive function to remove trailing zeros
        def remove_last_digit(digit):
            if digit and digit[-1]=='0':
                return remove_last_digit(digit[:-1])
            return digit if digit else '0'

        #reverses digits
        def reverse(digit):
            reverse = digit[::-1]
            return reverse

        #Looks at last digit decides if negative or positive, implements functions from above
        for dig in digits:
            dig = str(dig)
            if dig and dig[-1]=='0':
                dig = remove_last_digit(dig)
                rev = reverse(dig)
                dig=int(rev) * -1
                correct_order.append(int(dig))
            elif dig and dig[-1]=='1':
                dig = dig[:-1]
                dig = remove_last_digit(dig)
                rev = reverse(dig)
                correct_order.append(int(rev))

        #Save cleaned reconstruction
        np.save(f'reconstructed/reconstructed_colours_{side[direction]}_{frame}_correct_order.npy', correct_order)
        
    

In [40]:
#Checking if reconstruct
sample = left[:533]
print('Original first 20 inputs into left Sound frame 420')
print(sample[0:20])
print('---------------------------------------------')
test = np.load(f'reconstructed/reconstructed_colours_left_420_correct_order.npy')
print('Reconstructed first 20 inputs into left Sound frame 420')
print(test[0:20])


Original first 20 inputs into left Sound frame 420
[ 2557  2484  2306  2030  1665  1231   755   264  -204  -612  -919 -1088
 -1116 -1034  -897  -747  -597  -445  -287  -134]
---------------------------------------------
Reconstructed first 20 inputs into left Sound frame 420
[ 2557  2484  2306  2030  1665  1231   755   264  -204  -612  -919 -1088
 -1116 -1034  -897  -747  -597  -445  -287  -134]


In [2]:
#To add the sound back to the video first putting sound back together
left_reconstructed = []
right_reconstructed=[]
for frame in range(420,451):
    left_audio = np.load(f'reconstructed/reconstructed_colours_left_{frame}_correct_order.npy')
    for number in left_audio:
        left_reconstructed.append(number)
    right_audio = np.load(f'reconstructed/reconstructed_colours_right_{frame}_correct_order.npy')
    for number in right_audio:
        right_reconstructed.append(number)

In [5]:
#Creating a WAV of the audio
import numpy as np
import wave

left_channel = np.array(left_reconstructed, dtype=np.int16)  
right_channel = np.array(right_reconstructed, dtype=np.int16)  

# Ensure both channels have the same length
assert len(left_channel) == len(right_channel), "Left and Right channels must be the same length"

# Stack into stereo format (interleave left and right channel)
stereo_audio = np.column_stack((left_channel, right_channel)).flatten()

# Save to WAV file
sample_rate = 16000 
with wave.open("reconstructed_audio.wav", "w") as wav_file:
    wav_file.setnchannels(2)  # Stereo
    wav_file.setsampwidth(2)  # 16-bit PCM
    wav_file.setframerate(sample_rate)
    wav_file.writeframes(stereo_audio.tobytes())

print("WAV file created: reconstructed_audio.wav")


WAV file created: reconstructed_audio.wav


In [7]:
#Playing the audio
import IPython.display as display

display.Audio('reconstructed_audio.wav')

The video has been tested by downloading from GitHub to test the audio and frames match the original video. The reconstucted video is saved under "reconstructed_video0.mp4". To create, the orginal frames were used so size has not changed however the audio for computaional time reasons was reduced to 16000 fps, however from listening this hasn't seemed to change the quality of the sound.

In [16]:
#Putting the audio and frames together
import subprocess

# The FFmpeg command to run
ffmpeg_command = [
    "ffmpeg",
    "-y",
    "-framerate", "30",
    "-start_number", "420",
    "-i", "frames/frame_%03d.png",
    "-i", "reconstructed_audio.wav",
    "-c:v", "libx264",
    "-pix_fmt", "yuv420p",
    "-r", "30",
    "-t", "1",
    "-c:a", "aac",
    "-b:a", "192k",
    "-shortest", 
    "reconstructed_video0.mp4"
]

# Run the command using subprocess
subprocess.run(ffmpeg_command)


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/aarch64-linux-gnu --incdir=/usr/include/aarch64-linux-gnu --arch=arm64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --en

CompletedProcess(args=['ffmpeg', '-y', '-framerate', '30', '-start_number', '420', '-i', 'frames/frame_%03d.png', '-i', 'reconstructed_audio.wav', '-c:v', 'libx264', '-pix_fmt', 'yuv420p', '-r', '30', '-t', '1', '-c:a', 'aac', '-b:a', '192k', '-shortest', 'reconstructed_video0.mp4'], returncode=0)

## Results Analysis

This research explores a audio-visual data embedding and reconstruction technique using Spiking Neural Networks (SNNs). A single second of video is first decomposed into individual frames and its corresponding audio track. 256 unique temporal spiking patterns are created, using incremental intensity for each pattern. 256 was chosen to represent each pixel. Similarly, ten distinct spiking patterns are generated for the audio data. The audio is separated into left and right channels, and a preliminary test is conducted using randomly generated numbers, which are encoded using the 10 unique patterns found and visualized as six raster plots. This confirms the encoding method's viability. Subsequently, the actual left and right audio channels are encoded and visualized as raster plots. To embed the audio into the video frames, pixel values of 9 or below are replaced with 10, creating a marker. Unique audio values are also mapped to 10. The audio data is then embedded within the red, green, and blue color channels of pixels with the value 10, with each channel storing specific portions of the audio data. An SNN is then used to visualize the resulting raster plot, demonstrating the potential of SNNs for data encryption and optimization. The embedded audio is successfully decoded by identifying pixel values below 10 and converting them back to their original form. This decoded audio is then transformed into a .WAV file, which, when combined with the video frames, reconstructs the original .MP4 video.

## Conclusions and Future work
Draw the final conclusions and discuss the future work

In conclusion, this coursework presented a challenging yet stimulating exploration into audio-visual data embedding and SNN-based pattern recognition. The successful reconstruction of the original video from embedded audio data demonstrated the potential of SNNs for data manipulation and encryption. 

Future work could involve leveraging SNNs in a biological context, potentially developing digital twins and implementing SNN architectures for targeted tumor removal. This ambition reflects my deep resonance with advancing SNN applications in impactful, real-world scenarios.

## Discuss the AI ethics
Discuss the AI ethics implications in terms of security, protection, accountability, and algorithmic bias.

This coursework demonstrates powerful encryption and space optimization tools, with potential applications like securing patient data in medical imaging. These techniques offer significant strengths, including enhanced data security, efficient storage, and faster transmission. However, they also present weaknesses, such as potential vulnerabilities to sophisticated decryption attacks, the computational overhead associated with complex encryption algorithms, and the risk of data loss if decryption keys are compromised. It's crucial to acknowledge the dual nature of such technology; in the wrong hands, it could be used for harmful purposes, such as concealing sensitive military communications.

When considering AI ethics, particularly in the context of data encryption, four key principles are paramount:

Privacy- In scenarios like medical imaging, patient data must be encoded to prevent unauthorized access, ensuring confidentiality.

Transparency- Any encryption employed should be explicitly disclosed. This prevents the concealment of hidden messages or undisclosed data manipulation, fostering trust and openness.

Accountability- Developers must fully understand their code's implications, take responsibility for any errors, and be prepared to address them.

Bias Mitigation- Encryption and decryption methods must be designed to function consistently across all demographics. For example, when encoding patient data for both children and adults, the process should ensure no information loss or disparity in access.

In conclusion, while the presented encryption and optimization techniques offer significant benefits, particularly in secure data handling, their ethical deployment is paramount. The need for privacy, transparency, accountability, and bias mitigation is essential to ensure these powerful tools are used responsibly and for the greater good of society, rather than for potentially harmful purposes.

# The code above represents the effort and intellectual property of Ruth Amey. As an ethical principle, please respect this ownership and refrain from using, copying, or distributing this code without explicit permission.